In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
df =pd.read_csv('Churn_Modelling.csv')

In [3]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace =True)
X=df.drop(['Exited'],axis=1)
y=df['Exited']
num_features = X.select_dtypes(exclude="object").columns
onehot_columns=['Geography','Gender']
numeric_transformer= StandardScaler()
oh_transformer=OneHotEncoder()
preprocessor= ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,onehot_columns),
        ("StandardScaler",numeric_transformer,num_features)
    ],remainder='passthrough'
)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.42,random_state=42)
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [4]:
import pickle

In [5]:
with open("preprocessornew.pkl", "wb") as file:
    pickle.dump(preprocessor, file)

In [ ]:
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,
                    activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,
                        activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    opt=tf.keras.optimizers.Adam(learning_rate=0.01)

    model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])
    return  model

##create keras classifier
model=KerasClassifier(build_fn=create_model,epochs=50,batch_size=10,verbose=0) 

param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2,3,4],
    'epochs':[50,100]
}
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(X_train,y_train)

print("Best:%f using %s"(grid_result.best_score_,grid_result.best_params_))






<>:25: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:25: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?


AttributeError: 'super' object has no attribute '__sklearn_tags__'